<a href="https://colab.research.google.com/github/SevilAnna/GraduationProject/blob/main/EMU403_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>EMÜ 403 - Graduation Project</h1>

---

In [125]:
# importing libraries for the project
import pandas as pd
import numpy as np
from numpy.random import default_rng
rng = default_rng()
import random
import copy
!pip install geneticalgorithm
import geneticalgorithm as ga
from geneticalgorithm import geneticalgorithm as ga
print("done")

done


# Importing the Excel File

## Import file from PC


<font color='red'> 🔴🔴 Kızçeler, bir değişiklik yaptım kolay olsun diye 🔴🔴</font> 
</br>Drive'daki "Düzenlenmiş_Dişli_Parçaları" Excelini kendi bilgisayarınıza indirin. 
</br>Sonra veriyi çekmek için aşağıdaki kodu çalıştırın.

In [4]:
from google.colab import files
uploaded = files.upload()

Saving Düzenlenmiş_Dişli_Parçaları.xlsx to Düzenlenmiş_Dişli_Parçaları (1).xlsx


In [18]:
import io
df = pd.read_excel(io.BytesIO(uploaded["Düzenlenmiş_Dişli_Parçaları.xlsx"]), sheet_name='Anka', header=0)
del df['SÜRE (11 parça için)']
del df['1 tanesi için kaç tane gerek']
del df['PROSES']
del df['Process time']
df

,Makine,İşlem,SETUP,SÜRE (1 parça için),PARÇA
0,D13,Pre-Hardening Öncesi Torna,0.0,1.000000,UM121T900201200 GEAR ENGINE Z35
1,D33,Yıkama,0.0,0.090909,UM121T900201200 GEAR ENGINE Z35
2,D31,Ön Sertleştirme (Isıl İşlem),0.0,0.545455,UM121T900201200 GEAR ENGINE Z35
3,D33,Yıkama,0.0,0.090909,UM121T900201200 GEAR ENGINE Z35
4,D32,Temperleme (Isıl İşlem),0.0,0.454545,UM121T900201200 GEAR ENGINE Z35
...,...,...,...,...,...
315,D12,Yıkama,0.0,0.181818,UM121T900204300 GEAR ALTERNATOR Z13 -2
316,D69,Balans,0.0,1.000000,UM121T900204300 GEAR ALTERNATOR Z13 -2
317,D65,MPI,0.0,0.454545,UM121T900204300 GEAR ALTERNATOR Z13 -2
318,D12,Yıkama,0.0,0.090909,UM121T900204300 GEAR ALTERNATOR Z13 -2


# Next steps

In [19]:
m_list = np.unique(np.array(df["Makine"]))                          # Each machine needed for this piece
m_num_list = [1, 2, 1, 4, 3, 2, 3, 3, 3, 2, 2, 1, 1, 1, 2, 1, 1]    # How many there're of each machine (done manually now; automate later)
print(len(m_list),m_list)
print(len(m_num_list),m_num_list)

17 ['662' 'D11' 'D12' 'D13' 'D15' 'D17' 'D27' 'D31' 'D32' 'D33' 'D41' 'D43'
 'D44' 'D47' 'D65' 'D69' 'Nital Etche, Hydrojen Embr.']
17 [1, 2, 1, 4, 3, 2, 3, 3, 3, 2, 2, 1, 1, 1, 2, 1, 1]


In [126]:
# creating a dictionary with each machine (including duplicates)
all_machines = {}
n = 0
for m in m_list:
  m_num = m_num_list[n]
  if m_num > 0:
    all_machines[m] = []
    # Adding processes to the machines (but not to duplicates)
    for r in range(len(df["Makine"])):
      if m == df.iloc[r]["Makine"]:
        all_machines[m].append(r+1)
  # add duplicate machines without the processes
  if m_num > 1:
    m1 = m+"-1"
    all_machines[m1] = all_machines[m]
    all_machines.pop(m)
    if m_num > 1:
      m2 = m+"-2"
      all_machines[m2] = []
    if m_num > 2:
      m3 = m+"-3"
      all_machines[m3] = []
    if m_num > 3:
      m4 = m+"-4"
      all_machines[m4] = []   # later add more if there're 4+ of a certain machine ---> or just write a function for this 
  n += 1
   
for k in all_machines:
  print(k,"\t",all_machines[k])

662 	 [26, 32, 58, 64, 90, 96, 122, 128, 154, 160, 186, 192, 218, 224, 250, 256, 282, 288, 314, 320]
D11-1 	 [11, 23, 43, 55, 75, 87, 107, 119, 139, 151, 171, 183, 203, 215, 235, 247, 267, 279, 299, 311]
D11-2 	 []
D12 	 [13, 24, 28, 31, 45, 56, 60, 63, 77, 88, 92, 95, 109, 120, 124, 127, 141, 152, 156, 159, 173, 184, 188, 191, 205, 216, 220, 223, 237, 248, 252, 255, 269, 280, 284, 287, 301, 312, 316, 319]
D13-1 	 [1, 7, 33, 39, 65, 71, 97, 103, 129, 135, 161, 167, 193, 199, 225, 231, 257, 263, 289, 295]
D13-2 	 []
D13-3 	 []
D13-4 	 []
D15-1 	 [14, 20, 25, 46, 52, 57, 78, 84, 89, 110, 116, 121, 142, 148, 153, 174, 180, 185, 206, 212, 217, 238, 244, 249, 270, 276, 281, 302, 308, 313]
D15-2 	 []
D15-3 	 []
D17-1 	 [10, 22, 42, 54, 74, 86, 106, 118, 138, 150, 170, 182, 202, 214, 234, 246, 266, 278, 298, 310]
D17-2 	 []
D27-1 	 [21, 53, 85, 117, 149, 181, 213, 245, 277, 309]
D27-2 	 []
D27-3 	 []
D31-1 	 [3, 15, 35, 47, 67, 79, 99, 111, 131, 143, 163, 175, 195, 207, 227, 239, 259, 271, 29

In [134]:
machines = copy.deepcopy(all_machines)

# initialize genetic algorithm where each gene is a machine's schedule
num_genes = len(machines)
num_chro = 8
popu_size = (num_chro,num_genes)
new_popu = []

for i in range(num_chro):
  new_popu.append([machines])

# shuffle the order of processes in each machine/gene except for the first chromosome, and print results
first_one = True
for i in new_popu:
  if first_one == False:
    for j in i:
      for k in j:
        #print("---",j[k])
        temp_l = random.sample(list(j[k]),len(j[k]))
        j[k] = temp_l
        #print("-",j[k])
  first_one = False
  print(i)

[{'662': [26, 32, 58, 64, 90, 96, 122, 128, 154, 160, 186, 192, 218, 224, 250, 256, 282, 288, 314, 320], 'D11-1': [11, 23, 43, 55, 75, 87, 107, 119, 139, 151, 171, 183, 203, 215, 235, 247, 267, 279, 299, 311], 'D11-2': [], 'D12': [13, 24, 28, 31, 45, 56, 60, 63, 77, 88, 92, 95, 109, 120, 124, 127, 141, 152, 156, 159, 173, 184, 188, 191, 205, 216, 220, 223, 237, 248, 252, 255, 269, 280, 284, 287, 301, 312, 316, 319], 'D13-1': [1, 7, 33, 39, 65, 71, 97, 103, 129, 135, 161, 167, 193, 199, 225, 231, 257, 263, 289, 295], 'D13-2': [], 'D13-3': [], 'D13-4': [], 'D15-1': [14, 20, 25, 46, 52, 57, 78, 84, 89, 110, 116, 121, 142, 148, 153, 174, 180, 185, 206, 212, 217, 238, 244, 249, 270, 276, 281, 302, 308, 313], 'D15-2': [], 'D15-3': [], 'D17-1': [10, 22, 42, 54, 74, 86, 106, 118, 138, 150, 170, 182, 202, 214, 234, 246, 266, 278, 298, 310], 'D17-2': [], 'D27-1': [21, 53, 85, 117, 149, 181, 213, 245, 277, 309], 'D27-2': [], 'D27-3': [], 'D31-1': [3, 15, 35, 47, 67, 79, 99, 111, 131, 143, 163, 17

kizlar aklima bir fikir geldi.
bu listelerin sonuna idle time belirtecek bir seyler koysak ve onlari da shuffle etsek. idle time da bir variable cünkü.       

-sevil

# old version

In [31]:
a_list = np.unique(np.array(df["PARÇA"]))
num = df["PARÇA"].nunique(a_list[0])
u_num = [0]*num
for i in range(9):
  for item in df["PARÇA"]:
    if item == a_list[i]: 
      u_num[i] += 1
  print(u_num[i],"processes for piece",a_list[i])

32 processes for piece UM121T900201200 GEAR ENGINE Z35 
32 processes for piece UM121T900201600 GEAR INTERMEDIATE Z57
64 processes for piece UM121T900201700 GEAR Z51 
32 processes for piece UM121T900202100 GEAR PROPELLER 
32 processes for piece UM121T900203800 Z23 
32 processes for piece UM121T900203900 GEAR Z19 
32 processes for piece UM121T900203900 GEAR Z19 - 2
32 processes for piece UM121T900204300 GEAR ALTERNATOR Z13 
32 processes for piece UM121T900204300 GEAR ALTERNATOR Z13 -2


In [32]:
processes = []
for i in range(num):
  processes.append([])
#print(processes)

j = 0
k = 0
total = 0
for i in range(len(u_num)):
  total += u_num[i]
  while j < total:
    if j == total:
      k += 1
    #print(i,j,k,total,processes)
    #print(processes[i])
    processes[i].append(j+1)
    j += 1
  #print(total)

for i in processes:
  print(len(i),i)

32 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32]
32 [33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64]
64 [65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128]
32 [129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160]
32 [161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192]
32 [193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217,

In [33]:
def RandomList(nn, start = 0, end = 100):
    arr = []
    tmp = random.randint(start, end)
    for x in range(nn):
        while tmp in arr:
            tmp = random.randint(start, end)
        arr.append(tmp)  
    return arr

In [34]:
num_genes = total
num_chro = 8
popu_size = (num_chro,num_genes)
new_popu = []
for i in range(num_chro):
  new_popu.append([])
  new_popu[i] = RandomList(len(df["PARÇA"]),0,len(df["PARÇA"])-1)  #rng.choice(total+1, total+1, replace=False)

for i in new_popu:
  print(i)

[80, 230, 204, 27, 125, 180, 211, 202, 225, 192, 203, 280, 148, 294, 274, 210, 18, 84, 319, 118, 61, 38, 163, 62, 8, 103, 281, 311, 173, 36, 296, 90, 317, 264, 309, 20, 111, 86, 145, 253, 10, 169, 185, 268, 186, 96, 285, 241, 66, 198, 236, 56, 42, 57, 4, 193, 98, 2, 255, 217, 277, 49, 187, 263, 88, 291, 283, 112, 199, 152, 207, 7, 295, 256, 171, 200, 23, 233, 47, 124, 9, 297, 191, 151, 161, 79, 138, 251, 254, 282, 275, 287, 228, 45, 243, 166, 284, 65, 164, 216, 93, 68, 109, 170, 134, 205, 260, 188, 132, 246, 149, 52, 54, 33, 301, 59, 288, 234, 271, 25, 242, 178, 157, 195, 108, 190, 55, 249, 156, 144, 175, 37, 58, 189, 95, 177, 155, 153, 102, 89, 53, 13, 197, 146, 48, 127, 239, 67, 17, 270, 119, 261, 51, 313, 273, 77, 224, 250, 206, 237, 142, 32, 6, 26, 179, 262, 286, 150, 307, 293, 162, 15, 140, 183, 121, 160, 24, 304, 34, 257, 28, 70, 16, 5, 303, 266, 244, 232, 117, 74, 107, 147, 40, 305, 30, 143, 63, 99, 137, 176, 240, 114, 122, 115, 226, 218, 316, 81, 71, 154, 208, 227, 14, 126, 83,

In [117]:
num_genes = total
num_chro = 8
popu_size = (num_chro,num_genes)
new_popu = []
for i in range(num_chro):
  new_popu.append([])
  new_popu[i] = RandomList(len(df["PARÇA"]),0,len(df["PARÇA"])-1)  #rng.choice(total+1, total+1, replace=False)

for i in new_popu:
  print(i)

[276, 309, 99, 42, 315, 124, 23, 125, 228, 120, 95, 285, 313, 4, 149, 102, 143, 59, 186, 174, 263, 260, 103, 279, 7, 204, 57, 109, 129, 142, 50, 16, 220, 205, 284, 306, 176, 37, 151, 67, 169, 239, 231, 210, 62, 66, 68, 246, 307, 27, 319, 266, 166, 277, 314, 245, 58, 51, 213, 144, 184, 252, 248, 197, 65, 146, 191, 163, 9, 198, 310, 187, 90, 172, 61, 55, 111, 80, 77, 150, 305, 233, 118, 138, 140, 178, 214, 287, 29, 93, 106, 33, 286, 30, 225, 271, 72, 85, 135, 75, 41, 201, 253, 179, 108, 254, 49, 152, 10, 312, 78, 91, 87, 25, 21, 232, 159, 281, 183, 148, 43, 223, 199, 137, 112, 28, 121, 237, 250, 100, 291, 156, 104, 105, 73, 216, 81, 145, 269, 304, 155, 217, 24, 247, 194, 69, 265, 31, 188, 116, 97, 26, 270, 224, 17, 316, 47, 177, 241, 298, 19, 38, 272, 209, 32, 256, 53, 261, 74, 318, 167, 193, 182, 128, 240, 133, 229, 238, 234, 132, 157, 40, 44, 39, 160, 1, 64, 308, 18, 171, 45, 289, 202, 35, 207, 317, 219, 96, 195, 161, 215, 158, 8, 295, 126, 297, 303, 130, 274, 12, 236, 63, 259, 101, 24

# other

In [ ]:
'''class Process:
  def __init__(self, parameters):
    self.v = []

  def add_var(self, start_t, fin_t, wait_t):
    self.v.append(start_t, fin_t, wait_t)#+

  def add_par(self, B, S, setup_t, p_dur, T):
    self.v.append(B, S, setup_t, p_dur, T)

In [ ]:
'''# Mathematical Model


#Model Parameters
B(i) = [0,1] #Whether machine m can process operation j of piece i. Shown as 1 if piece i’s operation j can be processed at machine m, 0 o.w.
S(i) = [] #Starting time of piece i, operation j, relative to machine m, when order k≠1
F = [] #Finishing time of piece i, operation j, relative to machine m, at order k
K(i) = [] #Setup time for piece i, operation j at machine m
D(i) = [] #Process duration of piece i, operation j at machine m
T(i) = [] #Total processing time for all operations of piece i
#Decision Variables
X(i) = [0,1] #1 if piece i’s job j is processed at machine m’s kth order, 0 o.w.
S(i) = [] #Starting time of the first order (k=1) relative to and at machine m, for piece i, operation j 
W(i) = [] #Waiting time of piece i, operation j, at machine m, for order k
#Objective Function
obj_func = min(max(T))
# Constraints
S>=0
W>=0

For i in F:
  T(i) = sum(F) 


In [ ]:
'''import numpy

# This project is extended and a library called PyGAD is released to build the genetic algorithm.
# PyGAD documentation: https://pygad.readthedocs.io
# Install PyGAD: pip install pygad
# PyGAD source code at GitHub: https://github.com/ahmedfgad/GeneticAlgorithmPython

def cal_pop_fitness(equation_inputs, pop):
    # Calculating the fitness value of each solution in the current population.
    # The fitness function caulcuates the sum of products between each input and its corresponding weight.
    
    #tot_time = # sum() # + tot_waiting_time
    
    fitness = numpy.sum(pop*equation_inputs, axis=1)
    return fitness

def select_mating_pool(pop, fitness, num_parents):
    # Selecting the best individuals in the current generation as parents for producing the offspring of the next generation.
    parents = numpy.empty((num_parents, pop.shape[1]))
    for parent_num in range(num_parents):
        max_fitness_idx = numpy.where(fitness == numpy.max(fitness))
        max_fitness_idx = max_fitness_idx[0][0]
        parents[parent_num, :] = pop[max_fitness_idx, :]
        fitness[max_fitness_idx] = -99999999999
    return parents

def crossover(parents, offspring_size):
    ##### order based crossover technique, Partially mapped crossover (PMX) gerekli
    
    offspring = numpy.empty(offspring_size)
    # The point at which crossover takes place between two parents. Usually it is at the center.
    crossover_point = numpy.uint8(offspring_size[1]/2)

    for k in range(offspring_size[0]):
        # Index of the first parent to mate.
        parent1_idx = k%parents.shape[0]
        # Index of the second parent to mate.
        parent2_idx = (k+1)%parents.shape[0]
        # The new offspring will have its first half of its genes taken from the first parent.
        offspring[k, 0:crossover_point] = parents[parent1_idx, 0:crossover_point]
        # The new offspring will have its second half of its genes taken from the second parent.
        offspring[k, crossover_point:] = parents[parent2_idx, crossover_point:]
    return offspring

def mutation(offspring_crossover):
    # Mutation changes a single gene in each offspring randomly.
    for idx in range(offspring_crossover.shape[0]):
        # The random value to be added to the gene.
        random_value = numpy.random.uniform(-1.0, 1.0, 1)
        offspring_crossover[idx, 4] = offspring_crossover[idx, 4] + random_value
    return offspring_crossover